# A3 - cs2951-R - Motion Classification - Main
Shaun Wallace Files: features.py, classifier.py (Python 3.5.1 & Anaconda 4, Mac OS X 10.9.5)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn import datasets

from sklearn import tree, svm, neighbors, cross_validation, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

##### Datasets & Initial Features
The initial raw data was gathered using AndroSensor on Android.  For each activity my phone was placed in my pocket with the screen facing inwards towards my leg.  I gathered 7 to 15 minutes of data per activity.  The beggining and end of the data was truncated to create 5 minutes of data per acitivity file.  To assess the possible features each column was compared to elapsed time.  I felt to assess the data it was easiest to view the in this context.  After reviewing these relationships I felt most data was relevant to utilize to test features.  In my processing of data I utilized every combination of XYZ coordinates for each sensor.  For each column min, max, standard deviation, and mean was computed for each 10 seconds worth of data.  

A very enlighting view was looking at the testing data.  Which consited of 1 minute snippets for all activities.  This helps paint the picture about how different each activity was. Because of this I did not initially scale the data between -1 and 1.  This is because the range of the data and average value can vary greatly between each activity type: 

![title](img/accXtime.png)

##### Features

Originally I went through and computer min,max,stdev,mean, and used the resultant acceleration formula on XYZ axis for all 18 sensors.  I realized later on this was classic case of overfitting even though. My original accuracy scores were LogReg .97, Tree .98, Nearest Neighbors .935, and SVM .98.  While these were good numbers they were more a product of overfitting.  I could also correctly predict the activity in all instances except when walking and running got confused.  In order to narrow down my number features I went back to view plots of values vs time.

I went through the graphs and selected several features I thought could be potentially useful:
1. Acceleration X (min, max, stdev, mean)
2. Average Resultant Acceleration - (Acceleration XYZ) (np.sqrt(X+Y+Z))/100.0 <-X = sum(X) * sum(X)
3. Gravity Y (mean)
4. Linear Acceleration Z (min, max)
5. Average Resultant Linear Acceleration - (Linear Acceleration XYZ) (np.sqrt(X+Y+Z))/100.0 <-X = sum(X) * sum(X)
6. Gyroscope Y (stdev, max-min)
7. Gyroscope Z (min, max, stdev, mean, max-min)
8. Orientation Z (min, max, mean, max-min)

*max-min was scaled for only positive values

For my testing and iteration I utilized all 4 classifiers.  We only needed to use 2 of Decision Tree, Nearest Neighbor, and Support Vector Machines.  Utilizing all 4 helped to give me more information as I tested out various features and the relationships between the data.  My final selected feature set scores 100% accuracy with the required amount of classifiers laid out in the assignment.

Now my goal was to get the best accuracy ratings with using the least amount of features.  As I started to narrow down my features, initially working with groups of 10 my prediction scores were perfect:


In [3]:
activities = ['Driving', 'Running2', 'Stairs', 'Walking', 'WebBrowsing']
directory = '_DataFeaturesSmall/'

def featuresXY(filename):
    x = ''
    y = [filename] * 24
    xTest = ''
    yTest = [filename] * 6
    try:
        data = np.genfromtxt((directory+filename+'.csv'), delimiter=',')
        x = data[0:24,:]
        xTest = data[24:30,:]
    except Exception as e:
        print('Exception featuresXY: ', str(e))
    finally:
        return x, y, xTest, yTest


#Run through data...literally
Xdriving, Ydriving, XdrivingTest, YdrivingTest = featuresXY(activities[0])
Xrunning, Yrunning, XrunningTest, YrunningTest = featuresXY(activities[1]) #Running.csv I was wearing different pants
Xstairs, Ystairs, XstairsTest, YstairsTest = featuresXY(activities[2])
Xwalking, Ywalking, XwalkingTest, YwalkingTest = featuresXY(activities[3])
Xweb, Yweb, XwebTest, YwebTest = featuresXY(activities[4])


try:
    XtrainMast = np.array(Xdriving)
    XtrainMast = np.append(XtrainMast, Xrunning, 0)
    XtrainMast = np.append(XtrainMast, Xstairs, 0)
    XtrainMast = np.append(XtrainMast, Xwalking, 0)
    XtrainMast = np.append(XtrainMast, Xweb, 0)

    YtrainMast = np.array(Ydriving)
    YtrainMast = np.append(YtrainMast, Yrunning, 0)
    YtrainMast = np.append(YtrainMast, Ystairs, 0)
    YtrainMast = np.append(YtrainMast, Ywalking, 0)
    YtrainMast = np.append(YtrainMast, Yweb, 0)

    XtestMast = np.array(XdrivingTest)
    XtestMast = np.append(XtestMast, XrunningTest, 0)
    XtestMast = np.append(XtestMast, XstairsTest, 0)
    XtestMast = np.append(XtestMast, XwalkingTest, 0)
    XtestMast = np.append(XtestMast, XwebTest, 0)

    YtestMast = np.array(YdrivingTest)
    YtestMast = np.append(YtestMast, YrunningTest, 0)
    YtestMast = np.append(YtestMast, YstairsTest, 0)
    YtestMast = np.append(YtestMast, YwalkingTest, 0)
    YtestMast = np.append(YtestMast, YwebTest, 0)
except Exception as e:
    print('Exception append Arrays: ', str(e))
    
#Classifiers
clfLog = LogisticRegression().fit(XtrainMast,YtrainMast)
clfTree = tree.DecisionTreeClassifier().fit(XtrainMast,YtrainMast)
clfNbrs = KNeighborsClassifier(n_neighbors=1).fit(XtrainMast,YtrainMast)
clfSVM = svm.SVC(kernel='linear', C=2).fit(XtrainMast,YtrainMast)

#run basic scores and prediction
try:
    for act in activities:
        if act == activities[0]:
            xTest,yTest = XdrivingTest,YdrivingTest
        elif act == activities[1]:
            xTest,yTest = XrunningTest,YrunningTest
        elif act == activities[2]:
            xTest,yTest =  XstairsTest,YstairsTest
        elif act == activities[3]:
            xTest,yTest = XwalkingTest,YwalkingTest
        elif act == activities[4]:
            xTest,yTest = XwebTest,YwebTest

        print(act,' Basic Scores')
        print('log   = ', clfLog.score(xTest, yTest), clfLog.predict(xTest))
        print('tree  = ', clfTree.score(xTest, yTest), clfLog.predict(xTest))
        print('nbrs  = ', clfNbrs.score(xTest, yTest), clfLog.predict(xTest))
        print('svm   = ', clfSVM.score(xTest, yTest), clfLog.predict(xTest))
        print('')
except Exception as e:
    print('Exception runBasicScores: ', str(e))

Driving  Basic Scores
log   =  1.0 ['Driving' 'Driving' 'Driving' 'Driving' 'Driving' 'Driving']
tree  =  1.0 ['Driving' 'Driving' 'Driving' 'Driving' 'Driving' 'Driving']
nbrs  =  1.0 ['Driving' 'Driving' 'Driving' 'Driving' 'Driving' 'Driving']
svm   =  1.0 ['Driving' 'Driving' 'Driving' 'Driving' 'Driving' 'Driving']

Running2  Basic Scores
log   =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']
tree  =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']
nbrs  =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']
svm   =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']

Stairs  Basic Scores
log   =  1.0 ['Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs']
tree  =  1.0 ['Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs']
nbrs  =  1.0 ['Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs']
svm   =  1.0 ['Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs' 'Stairs']

Walking  Basic Scores
log   =  

These basic predictions are spot on. Once I got down to using less features I was predicting the activity with 100% accuracy using these basic means.  As I tried out different features 3 features consistently showed the most value:
1. Acceleration X - Standard Deviation
1. Acceleration X - Mean
3. Average Resultant Acceleration (Acceleration X)

Removing any of these three features decreased my scores.  I even tried Acceraltion Y and Z as well.  Those sensors often dropped my accuracy by 3-6 % points.  Since my phone was in my pocket facing the exact same direction for every activity it is not surpising one axis can so accuralety predict the data.

* *Running the plot3D() code below can lock up the Python Kernel in Jupyter Notebook*

In [ ]:
def plot3D(title, Xlabel, Ylabel, Zlabel, inc, i):
    #inc = 4 to go through XYZ for given sensor
    #inc = 13 go through avg resultant acceleration
    #i = starting point
    ii = i + inc
    iii = ii = inc
    Xdr = np.genfromtxt((directory+activities[0]+'.csv'), delimiter=',')
    Xru = np.genfromtxt((directory+activities[1]+'.csv'), delimiter=',')
    Xst = np.genfromtxt((directory+activities[2]+'.csv'), delimiter=',')
    Xwa = np.genfromtxt((directory+activities[3]+'.csv'), delimiter=',')
    Xwe = np.genfromtxt((directory+activities[4]+'.csv'), delimiter=',')
    try:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        #print(Xdr[:, 1])
        xa = Xdr[:, i]
        ya = Xdr[:, ii]
        za = Xdr[:, iii]

        xb = Xru[:, i]
        yb = Xru[:, ii]
        zb = Xru[:, iii]

        xc = Xst[:, i]
        yc = Xst[:, ii]
        zc = Xst[:, iii]

        xd = Xwa[:, i]
        yd = Xwa[:, ii]
        zd = Xwa[:, iii]

        xe = Xwe[:, i]
        ye = Xwe[:, ii]
        ze = Xwe[:, iii]

        dr = ax.scatter(xa, ya, za, c='red', marker='o', label='Driving')
        ru = ax.scatter(xb, yb, zb, c='orange', marker='o', label='Running')
        st = ax.scatter(xc, yc, zc, c='green', marker='o', label='Stairs')
        wa = ax.scatter(xd, yd, zd, c='purple', marker='o', label='Walking')
        we = ax.scatter(xe, ye, ze, c='blue', marker='^', label='WebBrowsing')

        ax.set_title(title)
        ax.set_xlabel('X '+ Xlabel)
        ax.set_ylabel('Y '+ Ylabel)
        ax.set_zlabel('Z '+ Zlabel)

        plt.legend(handles=[dr, ru, st, wa, we], loc=3)
        plt.show()
    except Exception as e:
        print('Exception plot3D: ', str(e))
    finally:
        return

plot3D('Features 3D Plot', 'AccelX Stdev', 'AccelX Mean', 'Avg Resultant Accel (AccelX)', 1, 0) #inc, i

##### Static Image of 3D Plot 
*(sometimes the 3D plot takes too long to execute in Jupyter Notebook)*
![title](img/3d1.png)

##  Cross Validation + Confusion Matrix

The purpose for cross-validation is to ascertain any tuning parameters and to help avoid overfitting.  Due to our initial high number of experimental features it is necessary to go through these steps to see the usefullness of our features.  It will also allow us to train on all activities at once.  We can then test with another array containing 1 minute snippets from each activity. (*The testing data was removed from the training data) 

Next comes slightly tuning classifiers after reviewing the 3D plot.  As you can see above the data sits relatively in its own space.  However Driving and Web Browser; and Stairs and Walking are close to each other.  I was walking up a hill for my walking data.  Because of this it is necessary to increase the granularity of the Nearest Neighbors and SVM classifiers before we feed them into the Cross Validation and Confusion Matrix.

Because I used a value of n_neighbors=1 for the cross validation.  Values of n_neighbors=5 worked perfectly fine for basic prediction.  However during my testing and data analysis to get 100% accuracy a more fine grained value for n_neighbors was needed.  The same logic applies to tweaking the C value for the Support Vector Machine classifier.  The larger the C valye the smaller the margin of the hyperplane used to slie and classify the data.  

The C parameter tells the SVM optimization how much you want to avoid misclassifying each training example. For large values of C, the optimization will choose a smaller-margin hyperplane if that hyperplane does a better job of getting all the training points classified correctly. Conversely, a very small value of C will cause the optimizer to look for a larger-margin separating hyperplane, even if that hyperplane misclassifies more points. For very tiny values of C, you should get misclassified examples, often even if your training data is linearly separable.


*The scores below use just Acceleration X - Standard Deviation, Acceleration X - Mean, and Average Resultant Acceleration (Acceleration X).*

```
Logistic Regression Confusion Matrix
Scores:  [ 0.92  1.    1.    1.    1.  ]
Accuracy Metrics: 0.9833
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 0.9840 (+/- 0.0640)

Decision Tree Confusion Matrix
Scores:  [ 1.    1.    0.96  1.    1.  ]
Accuracy Metrics: 0.9917
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 0.9920 (+/- 0.0320)

Nearest Neighbors Confusion Matrix
Scores:  [ 0.96  1.    1.    1.    1.  ]
Accuracy Metrics: 0.9917
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 0.9920 (+/- 0.0320)

Support Vector Machines Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 1.0000
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 1.0000 (+/- 0.0000)
```

My goal was to use as few features as possible.  While these three features are close they are not quite 100%.  I went through each of my preselected features adding 1 at time to see if they would positively effect the scores.  If this did not work I was goint to implement Some of my FFT code from breathing.py.  Almost all of the 4th features added had a negative effect on the accuracy scores.  However two improved the scores.  
1. Gravity Y (mean)
2. Gyroscope Y (stdev)


##### Gravity Y (mean) as 4th Feature
```
Logistic Regression Confusion Matrix
Scores:  [ 0.92  1.    1.    1.    1.  ]
Accuracy Metrics: 0.9833
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 0.9840 (+/- 0.0640)

Decision Tree Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 0.9917
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 1.0000 (+/- 0.0000)

Nearest Neighbors Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 1.0000
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 1.0000 (+/- 0.0000)

Support Vector Machines Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 1.0000
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 1.0000 (+/- 0.0000)
```

##### Gyroscope Y (stdev) as 4th Feature (BEST)

In [5]:
import warnings
warnings.filterwarnings('ignore')

def confusionMatrix(clfName, clf, Xtrain, Xtest, Ytrain, Ytest, color):
    try:
        clfRun = clf.fit(Xtrain, Ytrain).predict(Xtest)
        scores = cross_validation.cross_val_score(clf, Xtrain, Ytrain, cv=5)

        predicted = cross_validation.cross_val_predict(clf, Xtrain, Ytrain, cv=5)
        acc = metrics.accuracy_score(Ytrain, predicted)

        cm = confusion_matrix(Ytest, clfRun)
        cmNorm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        plt.figure() #
        plt.imshow(cmNorm, interpolation='nearest', cmap=color)
        plt.title(clfName), plt.colorbar()
        tick_marks = np.arange(len(activities))
        plt.xticks(tick_marks, activities, rotation=45)
        plt.yticks(tick_marks, activities)
        plt.tight_layout()
        plt.ylabel('True'), plt.xlabel('Predicted')
        print(clfName +  ' Confusion Matrix')
        print('Scores: ',scores)
        print('Accuracy Metrics: %0.4f' % (acc))
        print('CM Norm: ',cmNorm)
        print ('Accuracy CrossValScore: %0.4f (+/- %0.4f)' % (scores.mean(), scores.std() * 2))
        print('')
    except Exception as e:
        print('Exception confusionMatrix: ', str(e))
        
#refresh clf for confusion matrix
clfLog = LogisticRegression()
clfTree = tree.DecisionTreeClassifier()
clfNbrs = KNeighborsClassifier(n_neighbors=1)
clfSVM = svm.SVC(kernel='linear', C=2)
        
confusionMatrix('Logistic Regression', clfLog, XtrainMast, XtestMast, YtrainMast, YtestMast, plt.cm.Blues)
confusionMatrix('Decision Tree', clfTree, XtrainMast, XtestMast, YtrainMast, YtestMast, plt.cm.Reds)
confusionMatrix('Nearest Neighbors', clfNbrs, XtrainMast, XtestMast, YtrainMast, YtestMast, plt.cm.Oranges)
confusionMatrix('Support Vector Machines', clfSVM, XtrainMast, XtestMast, YtrainMast, YtestMast, plt.cm.Greens)

Logistic Regression Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 1.0000
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 1.0000 (+/- 0.0000)

Decision Tree Confusion Matrix
Scores:  [ 1.    1.    0.96  1.    1.  ]
Accuracy Metrics: 0.9917
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 0.9920 (+/- 0.0320)

Nearest Neighbors Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 1.0000
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
Accuracy CrossValScore: 1.0000 (+/- 0.0000)

Support Vector Machines Confusion Matrix
Scores:  [ 1.  1.  1.  1.  1.]
Accuracy Metrics: 1.0000
CM Norm:  [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  

##### Confusion Matrix Graphics:
Acceleration X - Standard Deviation, Acceleration X - Mean, 
Average Resultant Acceleration (Acceleration X), Gyroscope Y (stdev) 

![title](img/log.png)
![title](img/tree.png)
![title](img/nbrs.png)
![title](img/svm.png)

##### Final Analysis

The assignment asked for us to use Logisitic Regression and two of the other classifiers.  I believe Gyroscope Y (stdev) was the best 4th feature to add because it fulfilled the project requirements.  It scored 100% on 3 required classifiers.  While Gravity Y (mean) is an interesting feature it did not score 100% on the requied Logistic Regression.  Its lowest accuracy of .9840 is lower than Gyroscope Y (stdev)'s lowest accuracy score of .9920.  Hence while it is close it is not the best.  

Gyroscope Y makes sense as a good feature.  It captures the rotation and fine grained rotations of your leg.  It helps to capture these rotations for more exagerated movements such as Running, Stairs, and Walking.  It also helps distinguish Driving from Web Browsing.  Both are sendetary actions.  However driving at high speeds, such as my commute home work, generates distinct rotations that the Gyroscope is better equipped to capture than the accelerometer.  It was these fine grained differences the Accelerometer alone could not capture.  Hence why adding the Gyroscope helped boost the scores.

I tried using Gravity Y (mean) and Gyroscope Y (stdev) as 4th and 5th features.  However the accuracy scores were identical ot just using Gravity Y (mean). Hence I stuck with only 4 features with Gyroscope Y (stdev) as the 4th one.  Less really is more in this case.  On a side note, Average Resultant Linear Acceleration and Average Resultant Gyroscope readings also proved to be effective as 4th features, but not as much as the other two.  

### Pants Prediction + Analysis 

Pants are important:

*"Easy, guys.. I put my pants on just like the rest of you -- one leg at a time. Except, once my pants are on, I make gold records."* (Christopher Walken)


Now my Running2.csv was used instead of Running.csv because in Running2 I wore my khaki work pants I wore while collecting the other data.  For Running I wore Gym Shorts.  Which caused my phone to have more exagerated movements as I ran.  I ran the same analysis I ran before, replacing Driving with Running.  It appears from the data my classifer can distinguish between similar activities where I was wearing different pants:

```
Running  Basic Scores
log   =  1.0 ['Running' 'Running' 'Running' 'Running' 'Running' 'Running']
tree  =  1.0 ['Running' 'Running' 'Running' 'Running' 'Running' 'Running']
nbrs  =  1.0 ['Running' 'Running' 'Running' 'Running' 'Running' 'Running']
svm   =  1.0 ['Running' 'Running' 'Running' 'Running' 'Running' 'Running']

Running2  Basic Scores
log   =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']
tree  =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']
nbrs  =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']
svm   =  1.0 ['Running2' 'Running2' 'Running2' 'Running2' 'Running2' 'Running2']

SVM Cross Validation
Scores:  [ 0.96  1.    1.    1.    1.  ]
Accuracy:  0.9920 (+/- 0.0320)

Decision Tree Cross Validation
Scores:  [ 0.96  1.    1.    1.    0.95]
Accuracy:  0.9820 (+/- 0.0445)

Nearest Neighbors Cross Validation
Scores:  [ 0.96  1.    1.    1.    1.  ]
Accuracy:  0.9920 (+/- 0.0320)

SVM Cross Validation
Scores:  [ 0.96  1.    1.    1.    1.  ]
Accuracy:  0.9920 (+/- 0.0320)
```

In cross validation wearing gym shorts did cause some error but the standard deviation for my gym shorts data was incredibly small.  Overally I would say the classifiers and features can distinguish between what pants I was weaing.

### Notes + Bugs

**Running vs Runnng2 Data**

In Running I was wearing gym shorts with my phone in my right pocket.  This caused the phone to move wildly.  In Running2 I was wearing the same pants I worke for all other acitivites.  My phone in my right pocket.  


**Walking vs Walking2 Data**

Walking was taken while walking to class.  It was partially up hill.  Walking2 was taken when I walked from my house to a coffee shop on April 11th.  I just recorded the data to have some extra data to look at it.


**Possible Warnings**
/Applications/anaconda/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
  
I found this was an occasional error on Mac with matplotlib. It is nothing serious.  So it is perfectly okay if it pops up in the terminal while looking through the code.  In this notebook this code was used to hide the warnings:
```
import warnings
warnings.filterwarnings('ignore')
```